In [5]:
#import all required libraries
from configparser import ConfigParser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date
import pandas as pd



#Add User agent to scrape the site

service = ChromeService(executable_path=ChromeDriverManager().install())
options = ChromeOptions()


opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36")


In [49]:
driver = webdriver.Chrome(options=options, service=service)

driver.get('https://jailbreakbrewing.com')
button = driver.find_element(By.TAG_NAME, 'button')
button.click()
time.sleep(10)
jail_cols = driver.find_elements(By.CLASS_NAME,'whatsontaphdr')
jailbreak = pd.DataFrame(columns = ['Brewery','Date_Retrieved']+[col.text for col in jail_cols])
jail_beers = driver.find_elements(By.CLASS_NAME,'whatsontap')
i = 0
for col in jailbreak.columns[2:]:
    jailbreak[col] = [j.text for j in jail_beers[i::4]]
    i +=1
jailbreak['Brewery'] = 'Jailbreak Brewing'
jailbreak['Date_Retrieved'] = f"{date.today().month}/{date.today().day}/{date.today().year}"

In [46]:
driver.quit()

In [50]:
jailbreak

,Brewery,Date_Retrieved,Name,Style,ABV,IBU
0,Jailbreak Brewing,10/7/2022,Feed The Monkey,Orange Hefeweizen,5.6%,14
1,Jailbreak Brewing,10/7/2022,The Infinite,Amber Ale,5.0%,19
2,Jailbreak Brewing,10/7/2022,English Brown Ale,English Brown Ale,5.2%,N/A
3,Jailbreak Brewing,10/7/2022,Belgo Bleek,Belgian Pale Ale,5.5%,N/A
4,Jailbreak Brewing,10/7/2022,The Lesser Jesus,American Pale Ale,4.8%,30
5,Jailbreak Brewing,10/7/2022,Clearly Righteous,West Coast-Style IPA,6.7%,N/A
6,Jailbreak Brewing,10/7/2022,Dark & Righteous,Black IPA,6.5%,N/A
7,Jailbreak Brewing,10/7/2022,Hey porter,Robust Porter,6.5%,30
8,Jailbreak Brewing,10/7/2022,Special Lady Friend,Hazy IPA,7.0%,N/A
9,Jailbreak Brewing,10/7/2022,Doubles,Hazy DIPA,8.0%,N/A


In [51]:
driver.get("https://www.blackflagbrewingco.com")

In [52]:
button = driver.find_element(By.ID,"ageYes")
button.click()

In [53]:
elements = driver.find_elements(By.TAG_NAME,'th')
blackflag = pd.DataFrame(columns = ['Brewery','Date_Retrieved']+[e.text for e in elements])

In [54]:
blackflag

,Brewery,Date_Retrieved,Name,Type,Profile,ABV%,IBU


In [55]:
elements = driver.find_element(By.CLASS_NAME,'col-md-12')
flag_beers = elements.find_elements(By.TAG_NAME,'td')

i = 0
for col in blackflag.columns[2:]:
    blackflag[col] = [j.text for j in flag_beers[i::5]]
    i +=1
blackflag['Brewery'] = 'Black Flag Brewing'
blackflag['Date_Retrieved'] = f"{date.today().month}/{date.today().day}/{date.today().year}"



In [62]:
blackflag

,Brewery,Date_Retrieved,Name,Type,Profile,ABV%,IBU
0,Black Flag Brewing,10/7/2022,Space Nugs,IPA - New England / Hazy,Our latest cosmic conception is a 7% ABV hazy ...,7.0,35.0
1,Black Flag Brewing,10/7/2022,Cookie Dough Big Mage,Stout - Imperial / Double Pastry,After our anniversary party we decided to make...,14.0,0.0
2,Black Flag Brewing,10/7/2022,Chargin’ Mah Lazer,IPA - New England / Hazy,"A fresh 7% hazy IPA, Chargin’ Mah Lazer featur...",7.0,35.0
3,Black Flag Brewing,10/7/2022,A Wizard Is Sometimes Late,IPA - Imperial / Double New England / Hazy,A Wizard Is Sometimes Late is our 8% hazy DIPA...,8.0,35.0
4,Black Flag Brewing,10/7/2022,BLENDEE: Raspberry Cheesecake Graham Cracker,Sour - Fruited,Tart raspberry compliments the sour body right...,6.0,0.0
5,Black Flag Brewing,10/7/2022,Basic Bee Pumpkin Pie Ale,Pumpkin / Yam Beer,Vermont maple syrup. Wildflower and clover hon...,7.2,0.0
6,Black Flag Brewing,10/7/2022,Midnight Pumpkins,Stout - Coffee,We revamped a small batch fall favorite on the...,8.0,0.0
7,Black Flag Brewing,10/7/2022,Oktoberfest,Festbier,2021 Oktoberfest Marzen is here! This super tr...,5.6,10.0
8,Black Flag Brewing,10/7/2022,Flagship IPA,IPA - American,Citrus and Dank aromas and flavors paired with...,6.6,55.0
9,Black Flag Brewing,10/7/2022,King Kolsch,Kölsch,We brewed up an extremely easy drinking Kolsch...,5.0,10.0


In [63]:
driver.quit()